In [2]:
from xml.etree import ElementTree
from pynwb import NWBFile
from pathlib import Path

In [36]:
recfile = "../src/trodes_to_nwb/tests/test_data/20230622_sample_01_a1.rec"

new_recfile = Path('/stelmo/sam/truncated_rec_files/') / Path(recfile.split('/')[-1])
header_size = None
with open(recfile, mode="rb") as f:
    with open(new_recfile, mode="wb") as f2:
        for i in range(1000):
            line = f.readline()
            f2.write(line)
            



### Test can add ephys with new rec file

In [22]:
import os
import numpy as np
import pynwb
from trodes_to_nwb.convert_ephys import add_raw_ephys
from trodes_to_nwb import convert_yaml, convert_rec_header
from trodes_to_nwb.tests.test_convert_rec_header import default_test_xml_tree

MICROVOLTS_PER_VOLT = 1e6


path = "/home/sambray/Documents/trodes_to_nwb/src/trodes_to_nwb/tests"
metadata_path = path + "/test_data/20230622_sample_metadata.yml"
probe_metadata = [
    path + "/test_data/tetrode_12.5.yml",
]
metadata, probe_metadata = convert_yaml.load_metadata(metadata_path, probe_metadata)
nwbfile = convert_yaml.initialize_nwb(metadata, default_test_xml_tree())

# create the hw_channel map using the reconfig header
try:
    # running on github
    trodesconf_file = (
        os.environ.get("DOWNLOAD_DIR") + "/20230622_sample_01_a1.rec"
    )  # "/test_data/reconfig_probeDevice.trodesconf"
    rec_header = convert_rec_header.read_header(trodesconf_file)
except:
    # running locally
    trodesconf_file = (
        path + "/test_data/20230622_sample_01_a1.rec"
    )  # "/test_data/reconfig_probeDevice.trodesconf"
    rec_header = convert_rec_header.read_header(trodesconf_file)

hw_channel_map = convert_rec_header.make_hw_channel_map(
    metadata, rec_header.find("SpikeConfiguration")
)
ref_electrode_map = convert_rec_header.make_ref_electrode_map(
    metadata, rec_header.find("SpikeConfiguration")
)

convert_yaml.add_electrode_groups(
    nwbfile, metadata, probe_metadata, hw_channel_map, ref_electrode_map
)

try:
    # running on github
    recfile = os.environ.get("DOWNLOAD_DIR") + "/20230622_sample_01_a1.rec"
    rec_to_nwb_file = os.environ.get("DOWNLOAD_DIR") + "/20230622_155936.nwb"
except (TypeError, FileNotFoundError):
    # running locally
    recfile = path + "/test_data/20230622_sample_01_a1.truncated.rec"
    rec_to_nwb_file = path + "/test_data/20230622_155936.nwb"

map_row_ephys_data_to_row_electrodes_table = list(range(len(nwbfile.electrodes)))

add_raw_ephys(
    nwbfile,
    [
        recfile,
    ],
    map_row_ephys_data_to_row_electrodes_table,
)

/home/sambray/mambaforge-pypy3/envs/trodes_to_nwb/lib/python3.11/site-packages/pynwb/file.py:470: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)
